In [1]:
import http.client
import json
import pandas as pd

In [10]:
conn = http.client.HTTPSConnection("yt-api.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "e432b0e9efmshb5b9d62e77e42c1p14875ejsn117a50acc3c7",
    'x-rapidapi-host': "yt-api.p.rapidapi.com"
}

In [16]:
conn.request("GET", "/comments?id=1gxf6flnvNA", headers=headers)

res = conn.getresponse()
data = res.read()

In [21]:
parsed_data = json.loads(data)

parsed_data.keys()

dict_keys(['commentsCount', 'continuation', 'data', 'msg'])

In [23]:
data_list = parsed_data["data"]

In [24]:
comments_df = pd.DataFrame(data_list)

In [28]:
comments_df.head()

,commentId,authorText,authorChannelId,authorThumbnail,textDisplay,publishedTimeText,publishDate,publishedAt,likesCount,replyCount,replyToken,authorIsChannelOwner,isVerified,isArtist,isCreator
0,Ugwq5-FSu3zt7iVrMyp4AaABAg,@allaboutroofing2,UC1oyzRrNZ4YkJL3EufJkZEw,[{'url': 'https://yt3.ggpht.com/QITvE_yvcpQYbS...,Costco would sell a lot more large screen 4K T...,5 years ago,2019-06-11,2019-06-11T00:00:00Z,2.2K,20,Eg0SCzFneGY2Zmxudk5BGAYygwEaUBIaVWd3cTUtRlN1M3...,False,False,False,False
1,UgwKyG3jjt78nl0Uijt4AaABAg,@NotDeadYet..,UCUAP8M8Lm1AdCZ0uVJC6J3w,[{'url': 'https://yt3.ggpht.com/Q1Z2iBknL5mTKr...,The song is majestic\nThe cut and composition ...,5 years ago,2019-06-11,2019-06-11T00:00:00Z,682,3,Eg0SCzFneGY2Zmxudk5BGAYygwEaUBIaVWd3S3lHM2pqdD...,False,False,False,False
2,Ugyo69dfDTfBIJSKG_14AaABAg,@NeonblueIndustries,UC_ujj6ZV9SbD2rjs5-tl1iQ,[{'url': 'https://yt3.ggpht.com/ytc/AIdro_meiu...,One of the greatest works of cinematography in...,5 years ago,2019-06-11,2019-06-11T00:00:00Z,124,0,NaN,False,False,False,False
3,Ugw09y6I68Z7ZaliLqJ4AaABAg,@CarloGoiff666,UCZZQmlnCk8XbzJ9d2IlgEcQ,[{'url': 'https://yt3.ggpht.com/ytc/AIdro_lodK...,"""Director of photography/Director - Dustin Far...",5 years ago,2019-06-11,2019-06-11T00:00:00Z,654,2,Eg0SCzFneGY2Zmxudk5BGAYygwEaUBIaVWd3MDl5Nkk2OF...,False,False,False,False
4,UgyMUcMwxQFGgF2tCkV4AaABAg,@conyo985,UCa9Ze3yzu06uTEZb3evJv8w,[{'url': 'https://yt3.ggpht.com/ytc/AIdro_lFgB...,This is a great 4K demo for any 4K TV! It shou...,5 years ago,2019-06-11,2019-06-11T00:00:00Z,167,2,Eg0SCzFneGY2Zmxudk5BGAYygwEaUBIaVWd5TVVjTXd4UU...,False,False,False,False


In [33]:
sentence_list = []

for comment in comments_df["textDisplay"]:
    sentence_list.append(comment)

In [34]:
import torch
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertModel.from_pretrained('bert-base-uncased')


inputs = tokenizer(sentence_list, return_tensors='pt', padding=True, truncation=True, max_length=128)


with torch.no_grad():
    outputs = model(**inputs)


hidden_states = outputs.last_hidden_state


attention_mask = inputs['attention_mask']
sentence_embeddings = (hidden_states * attention_mask.unsqueeze(-1)).sum(1) / attention_mask.sum(1).unsqueeze(-1)

print(sentence_embeddings)

tensor([[ 0.1371,  0.1797,  0.3005,  ..., -0.2091,  0.5560, -0.0630],
        [ 0.0646, -0.2005,  0.6505,  ..., -0.4231, -0.0741, -0.2405],
        [-0.3408,  0.2411, -0.1606,  ..., -0.4825,  0.3362, -0.1827],
        ...,
        [ 0.1615, -0.4378,  0.6191,  ...,  0.0098,  0.2587, -0.5248],
        [ 0.3408,  0.0540,  0.0932,  ..., -0.0124,  0.1758, -0.2637],
        [ 0.1738,  0.1659,  0.2281,  ..., -0.2908,  0.0560, -0.0848]])


In [44]:
from sklearn.cluster import KMeans

num_clusters = 2  # You can adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(sentence_embeddings)

# Get cluster labels
labels = kmeans.labels_

# Print sentences grouped by cluster
clusters = {}
for label, sentence in zip(labels, sentence_list):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(sentence)

for cluster, sentences in clusters.items():
    print(f"Cluster {cluster}:")
    for sentence in sentences:
        print(f"  {sentence}")

Cluster 1:
  Costco would sell a lot more large screen 4K TVs  if they had this playing instead of those stupid parrots and lizards.  THIS is why we have 4K tv.
  The song is majestic
The cut and composition of the video godly
The girls bouncing around motorcyclist jump + firework + water bubble etc epic idea
This is 15/10
  One of the greatest works of cinematography in existence.
  This is a great 4K demo for any 4K TV! It should be shown in stores and it will drastically increase sales!
  This is my go to video for showing off my new 4k tv
  Once upon a time this was one of the few videos to test 4k, the 1st search result was this master piece
  Put this on my new Tv as a test made me smile! my wife not so much
  The amount of devices and skills involved in making this is the actual tribute!
From high speed cameras, lenses, lighting, water spraying devices, controlled flames and explosions, trampolines, motorcycles, breast implants, acrobatics, post production, know-how, etc. 
All t

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
